In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import multivariate_logrank_test

In [2]:
data_dir = '/your path/multiomics-cardiovascular-disease/data/'
results_dir = '/your path/multiomics-cardiovascular-disease/saved/results/'
log_dir = '/your path/multiomics-cardiovascular-disease/saved/log'
split_times = 10
seed_to_split = 241104
num_workers = 8

split_seed_filename = f"split_seed-{seed_to_split}/"
split_seed_dir = os.path.join(data_dir, split_seed_filename)

In [3]:
survival_prob = []

for k in range(10):
    
    genomics = pd.read_feather(split_seed_dir + f"X_test_Genomics_K{k}.feather")
    genomics.columns = [col.replace("prs_", "") if "prs_" in col else col for col in genomics.columns]
    genomics = genomics.rename(columns={"vte": "vt"})
    genomics['cvd_death'] = genomics['cad']
    genomics_long = pd.melt(genomics, id_vars=['eid'], var_name='outcome', value_name='prs')
    genomics_long['fold'] = k
    
    metabolomics = pd.read_csv(os.path.join(results_dir, f"MetScore/test_scores_K{k}.csv"))
    metabolomics_long = pd.melt(metabolomics, id_vars=['eid'], var_name='outcome', value_name='metscore')
    
    proteomics = pd.read_csv(os.path.join(results_dir, f"ProScore/test_scores_K{k}.csv"))
    proteomics_long = pd.melt(proteomics, id_vars=['eid'], var_name='outcome', value_name='proscore')
    
    e = pd.read_feather(os.path.join(split_seed_dir, f'e_test_K{k}.feather'))
    e_long = pd.melt(e, id_vars=['eid'], var_name='outcome', value_name='event')
    y = pd.read_feather(os.path.join(split_seed_dir, f'y_test_K{k}.feather'))
    y_long = pd.melt(y, id_vars=['eid'], var_name='outcome', value_name='time')
    
    merged_data = pd.merge(genomics_long, metabolomics_long, on=['eid', 'outcome'], how='left')
    merged_data = pd.merge(merged_data, proteomics_long, on=['eid', 'outcome'], how='left')
    merged_data = pd.merge(merged_data, e_long, on=['eid', 'outcome'], how='left')
    merged_data = pd.merge(merged_data, y_long, on=['eid', 'outcome'], how='left')
    # merged_data = pd.merge(merged_data, age_sex, on=['eid'], how='left')
    
    survival_prob.append(merged_data)
    
survival_prob = pd.concat(survival_prob, ignore_index=True)
survival_prob

,eid,outcome,prs,fold,metscore,proscore,event,time
0,1002627,cad,-0.513180,0,-2.913559,-3.038017,0,14.690411
1,1006647,cad,-1.513317,0,-3.728572,-3.064125,0,15.093151
2,1007634,cad,1.964256,0,-2.643847,-2.794146,0,15.169863
3,1010327,cad,-1.175524,0,-3.297099,-3.093149,0,15.709589
4,1013147,cad,0.632512,0,-3.445893,-2.924177,0,15.923288
...,...,...,...,...,...,...,...,...
218767,6018531,cvd_death,-1.381985,9,-4.466974,-4.955414,0,15.657534
218768,6021114,cvd_death,1.912248,9,-2.998327,-2.645683,0,14.134247
218769,6022027,cvd_death,-0.130757,9,-3.974407,-3.487737,0,14.134247
218770,6022766,cvd_death,0.543610,9,-4.498716,-4.900415,0,14.780822


In [4]:
survival_prob['prs_group'] = None
survival_prob['metscore_group'] = None
survival_prob['proscore_group'] = None

outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    subset = survival_prob[survival_prob['outcome'] == outcome]

    prs_low = subset['prs'].quantile(1/3)
    prs_high = subset['prs'].quantile(2/3)
    print(f"Outcome: {outcome} - PRS Tertiles: Low cutoff = {prs_low}, High cutoff = {prs_high}")
    
    metscore_low = subset['metscore'].quantile(1/3)
    metscore_high = subset['metscore'].quantile(2/3)
    print(f"Outcome: {outcome} - MetScore Tertiles: Low cutoff = {metscore_low}, High cutoff = {metscore_high}")
    
    proscore_low = subset['proscore'].quantile(1/3)
    proscore_high = subset['proscore'].quantile(2/3)
    print(f"Outcome: {outcome} - ProScore Tertiles: Low cutoff = {proscore_low}, High cutoff = {proscore_high}")

    survival_prob.loc[survival_prob['outcome'] == outcome, 'prs_group'] = pd.cut(
        survival_prob.loc[survival_prob['outcome'] == outcome, 'prs'],
        bins=[-float('inf'), prs_low, prs_high, float('inf')],
        labels=['low', 'medium', 'high']
    )
    
    survival_prob.loc[survival_prob['outcome'] == outcome, 'metscore_group'] = pd.cut(
        survival_prob.loc[survival_prob['outcome'] == outcome, 'metscore'],
        bins=[-float('inf'), metscore_low, metscore_high, float('inf')],
        labels=['low', 'medium', 'high']
    )

    survival_prob.loc[survival_prob['outcome'] == outcome, 'proscore_group'] = pd.cut(
        survival_prob.loc[survival_prob['outcome'] == outcome, 'proscore'],
        bins=[-float('inf'), proscore_low, proscore_high, float('inf')],
        labels=['low', 'medium', 'high']
    )
survival_prob

Outcome: cad - PRS Tertiles: Low cutoff = -0.4316869818580741, High cutoff = 0.41637004198433913
Outcome: cad - MetScore Tertiles: Low cutoff = -2.9879608666666666, High cutoff = -2.344820566666667
Outcome: cad - ProScore Tertiles: Low cutoff = -2.9696182333333336, High cutoff = -2.5539436000000006
Outcome: stroke - PRS Tertiles: Low cutoff = -0.441790932373319, High cutoff = 0.4281759234891326
Outcome: stroke - MetScore Tertiles: Low cutoff = -4.162642166666666, High cutoff = -3.6767324333333335
Outcome: stroke - ProScore Tertiles: Low cutoff = -3.8752082000000003, High cutoff = -3.6260561
Outcome: hf - PRS Tertiles: Low cutoff = -0.43829637753094497, High cutoff = 0.4289400053051556
Outcome: hf - MetScore Tertiles: Low cutoff = -4.0218861666666665, High cutoff = -3.358833433333334
Outcome: hf - ProScore Tertiles: Low cutoff = -3.9689972, High cutoff = -3.5013515333333336
Outcome: af - PRS Tertiles: Low cutoff = -0.44031982952855003, High cutoff = 0.406527495267634
Outcome: af - MetSc

,eid,outcome,prs,fold,metscore,proscore,event,time,prs_group,metscore_group,proscore_group
0,1002627,cad,-0.513180,0,-2.913559,-3.038017,0,14.690411,low,medium,low
1,1006647,cad,-1.513317,0,-3.728572,-3.064125,0,15.093151,low,low,low
2,1007634,cad,1.964256,0,-2.643847,-2.794146,0,15.169863,high,medium,medium
3,1010327,cad,-1.175524,0,-3.297099,-3.093149,0,15.709589,low,low,low
4,1013147,cad,0.632512,0,-3.445893,-2.924177,0,15.923288,high,low,medium
...,...,...,...,...,...,...,...,...,...,...,...
218767,6018531,cvd_death,-1.381985,9,-4.466974,-4.955414,0,15.657534,low,medium,low
218768,6021114,cvd_death,1.912248,9,-2.998327,-2.645683,0,14.134247,high,high,high
218769,6022027,cvd_death,-0.130757,9,-3.974407,-3.487737,0,14.134247,medium,medium,high
218770,6022766,cvd_death,0.543610,9,-4.498716,-4.900415,0,14.780822,high,medium,low


# Survival analyases for PRS

## KM curve

In [5]:
combined_data = []
groups_list = ['low', 'medium', 'high']
outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    for group in groups_list:
        kmf = KaplanMeierFitter()
        mask = (survival_prob['outcome'] == outcome) & (survival_prob['prs_group'] == group)
        kmf.fit(survival_prob[mask]['time'], survival_prob[mask]['event'])
        data = kmf.confidence_interval_survival_function_.copy()
        data['timeline'] = kmf.survival_function_.index
        data['survival_prob'] = kmf.survival_function_['KM_estimate']
        data['group'] = group
        data['outcome'] = outcome

        data = data.rename(columns={
            'KM_estimate_lower_0.95': 'lower_0.95',
            'KM_estimate_upper_0.95': 'upper_0.95'
        })

        combined_data.append(data)

combined_data = pd.concat(combined_data).reset_index(drop=True)
combined_data.to_csv('survival_prs.csv', index=False)
combined_data

,lower_0.95,upper_0.95,timeline,survival_prob,group,outcome
0,1.000000,1.000000,0.000000,1.000000,low,cad
1,0.999124,0.999983,0.024658,0.999877,low,cad
2,0.999013,0.999938,0.172603,0.999753,low,cad
3,0.998853,0.999881,0.210959,0.999630,low,cad
4,0.998853,0.999881,0.221918,0.999630,low,cad
...,...,...,...,...,...,...
45147,0.968711,0.976076,16.594521,0.972636,high,cvd_death
45148,0.968711,0.976076,16.610959,0.972636,high,cvd_death
45149,0.968711,0.976076,16.613699,0.972636,high,cvd_death
45150,0.968711,0.976076,16.616438,0.972636,high,cvd_death


### Log-rank test

In [6]:
logrank_df = []
outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    mask = (survival_prob['outcome'] == outcome)
    results = multivariate_logrank_test(survival_prob[mask]['time'], 
                                        survival_prob[mask]['prs_group'], 
                                        survival_prob[mask]['event'], weightings='wilcoxon')
    data = {
        'outcome': outcome,
        'score': 'prs',
        'p': results.p_value,
        'test_statistic': results.test_statistic
    }
    data = pd.DataFrame([data])
    
    logrank_df.append(data)

## Hazard ratio

In [7]:
hr_df = []
outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    subset = survival_prob[survival_prob['outcome'] == outcome][['prs_group', 'time', 'event']]
    subset = pd.get_dummies(subset, columns=['prs_group'], prefix='prs', drop_first=False)
    subset.drop(columns=['prs_low'], inplace=True)
    
    cox = CoxPHFitter(penalizer=0.03)
    cox.fit(subset, duration_col='time', event_col='event')
    data = cox.summary.reset_index().rename(columns={
        'index': 'variable',
        'coef': 'coef',
        'exp(coef)': 'exp(coef)',
        'se(coef)': 'se(coef)',
        'coef lower 95%': 'coef lower 95%',
        'coef upper 95%': 'coef upper 95%',
        'exp(coef) lower 95%': 'exp(coef) lower 95%',
        'exp(coef) upper 95%': 'exp(coef) upper 95%',
        'z': 'z',
        'p': 'p',
        '-log2(p)': '-log2(p)'
    })
    data['outcome'] = outcome

    hr_df.append(data)

hr_df = pd.concat(hr_df).reset_index(drop=True)
hr_df.to_csv('survival_hazard_ratio_prs.csv', index=False)
hr_df

,covariate,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p),outcome
0,prs_high,0.379568,1.461653,0.044823,0.291717,0.467418,1.338724,1.595869,0.0,8.468225,2.491597e-17,55.155707,cad
1,prs_medium,0.139054,1.149186,0.046479,0.047957,0.230151,1.049126,1.258790,0.0,2.991770,2.773652e-03,8.493998,cad
2,prs_high,0.138083,1.148071,0.059653,0.021166,0.255000,1.021392,1.290462,0.0,2.314793,2.062426e-02,5.599514,stroke
3,prs_medium,0.003825,1.003833,0.060475,-0.114703,0.122354,0.891631,1.130154,0.0,0.063256,9.495626e-01,0.074665,stroke
4,prs_high,0.027184,1.027557,0.056168,-0.082904,0.137272,0.920439,1.147140,0.0,0.483973,6.284051e-01,0.670233,hf
5,prs_medium,0.012481,1.012559,0.056268,-0.097802,0.122764,0.906829,1.130618,0.0,0.221816,8.244572e-01,0.278484,hf
6,prs_high,0.440900,1.554105,0.046988,0.348805,0.532995,1.417373,1.704029,0.0,9.383218,6.398916e-21,67.082662,af
7,prs_medium,0.142350,1.152981,0.048986,0.046340,0.238361,1.047431,1.269167,0.0,2.905955,3.661345e-03,8.093410,af
8,prs_high,-0.013479,0.986611,0.068988,-0.148694,0.121735,0.861833,1.129455,0.0,-0.195384,8.450923e-01,0.242819,va
9,prs_medium,0.020434,1.020644,0.068856,-0.114520,0.155389,0.891794,1.168112,0.0,0.296768,7.666436e-01,0.383372,va


# Survival analyases for MetScore

## KM curve

In [8]:
combined_data = []
groups_list = ['low', 'medium', 'high']
outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    for group in groups_list:
        kmf = KaplanMeierFitter()
        mask = (survival_prob['outcome'] == outcome) & (survival_prob['metscore_group'] == group)
        kmf.fit(survival_prob[mask]['time'], survival_prob[mask]['event'])
        data = kmf.confidence_interval_survival_function_.copy()
        data['timeline'] = kmf.survival_function_.index
        data['survival_prob'] = kmf.survival_function_['KM_estimate']
        data['group'] = group
        data['outcome'] = outcome

        data = data.rename(columns={
            'KM_estimate_lower_0.95': 'lower_0.95',
            'KM_estimate_upper_0.95': 'upper_0.95'
        })

        combined_data.append(data)

combined_data = pd.concat(combined_data).reset_index(drop=True)
combined_data.to_csv('survival_met.csv', index=False)
combined_data

,lower_0.95,upper_0.95,timeline,survival_prob,group,outcome
0,1.000000,1.000000,0.000000,1.000000,low,cad
1,0.999124,0.999983,0.115068,0.999877,low,cad
2,0.999013,0.999938,0.210959,0.999753,low,cad
3,0.999013,0.999938,0.443836,0.999753,low,cad
4,0.998852,0.999881,0.446575,0.999630,low,cad
...,...,...,...,...,...,...
44713,0.943814,0.954005,16.602740,0.949157,high,cvd_death
44714,0.943814,0.954005,16.610959,0.949157,high,cvd_death
44715,0.943814,0.954005,16.613699,0.949157,high,cvd_death
44716,0.943814,0.954005,16.616438,0.949157,high,cvd_death


### Log-rank test

In [9]:
outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    mask = (survival_prob['outcome'] == outcome)
    results = multivariate_logrank_test(survival_prob[mask]['time'], 
                                        survival_prob[mask]['metscore_group'], 
                                        survival_prob[mask]['event'], weightings='wilcoxon')
    data = {
        'outcome': outcome,
        'score': 'metscore',
        'p': results.p_value,
        'test_statistic': results.test_statistic
    }
    data = pd.DataFrame([data])
    
    logrank_df.append(data)

## Hazard ratio

In [10]:
hr_df = []
outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    subset = survival_prob[survival_prob['outcome'] == outcome][['metscore_group', 'time', 'event']]
    subset = pd.get_dummies(subset, columns=['metscore_group'], prefix='metscore', drop_first=False)
    subset.drop(columns=['metscore_low'], inplace=True)
    
    cox = CoxPHFitter(penalizer=0.03)
    cox.fit(subset, duration_col='time', event_col='event')
    data = cox.summary.reset_index().rename(columns={
        'index': 'variable',
        'coef': 'coef',
        'exp(coef)': 'exp(coef)',
        'se(coef)': 'se(coef)',
        'coef lower 95%': 'coef lower 95%',
        'coef upper 95%': 'coef upper 95%',
        'exp(coef) lower 95%': 'exp(coef) lower 95%',
        'exp(coef) upper 95%': 'exp(coef) upper 95%',
        'z': 'z',
        'p': 'p',
        '-log2(p)': '-log2(p)'
    })
    data['outcome'] = outcome

    hr_df.append(data)

hr_df = pd.concat(hr_df).reset_index(drop=True)
hr_df.to_csv('survival_hazard_ratio_met.csv', index=False)
hr_df

,covariate,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p),outcome
0,metscore_high,0.775614,2.171925,0.044833,0.687743,0.863484,1.989221,2.371409,0.0,17.300178,4.689373e-67,220.339787,cad
1,metscore_medium,0.200383,1.221870,0.048264,0.105788,0.294978,1.111586,1.343096,0.0,4.151839,3.298137e-05,14.887989,cad
2,metscore_high,0.364108,1.439230,0.059247,0.247987,0.480230,1.281443,1.616446,0.0,6.145637,7.964301e-10,30.225733,stroke
3,metscore_medium,0.020855,1.021074,0.061086,-0.098873,0.140582,0.905858,1.150943,0.0,0.341395,7.328065e-01,0.448496,stroke
4,metscore_high,0.635749,1.888436,0.055015,0.527921,0.743577,1.695404,2.103447,0.0,11.555826,6.898107e-31,100.193570,hf
5,metscore_medium,-0.002788,0.997215,0.058752,-0.117941,0.112364,0.888748,1.118920,0.0,-0.047462,9.621453e-01,0.055673,hf
6,metscore_high,0.615187,1.850002,0.046590,0.523871,0.706502,1.688552,2.026890,0.0,13.204163,8.301737e-40,129.823710,af
7,metscore_medium,0.066352,1.068603,0.050029,-0.031702,0.164407,0.968795,1.178693,0.0,1.326286,1.847450e-01,2.436393,af
8,metscore_high,0.175004,1.191251,0.068661,0.040431,0.309577,1.041259,1.362848,0.0,2.548814,1.080898e-02,6.531625,va
9,metscore_medium,0.031765,1.032275,0.069109,-0.103687,0.167216,0.901508,1.182010,0.0,0.459632,6.457806e-01,0.630884,va


# Survival analyases for ProScore

## KM curve

In [11]:
combined_data = []
groups_list = ['low', 'medium', 'high']
outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    for group in groups_list:
        kmf = KaplanMeierFitter()
        mask = (survival_prob['outcome'] == outcome) & (survival_prob['proscore_group'] == group)
        kmf.fit(survival_prob[mask]['time'], survival_prob[mask]['event'])
        data = kmf.confidence_interval_survival_function_.copy()
        data['timeline'] = kmf.survival_function_.index
        data['survival_prob'] = kmf.survival_function_['KM_estimate']
        data['group'] = group
        data['outcome'] = outcome

        data = data.rename(columns={
            'KM_estimate_lower_0.95': 'lower_0.95',
            'KM_estimate_upper_0.95': 'upper_0.95'
        })

        combined_data.append(data)

combined_data = pd.concat(combined_data).reset_index(drop=True)
combined_data.to_csv('survival_pro.csv', index=False)
combined_data

,lower_0.95,upper_0.95,timeline,survival_prob,group,outcome
0,1.000000,1.000000,0.000000,1.000000,low,cad
1,0.999124,0.999983,0.150685,0.999877,low,cad
2,0.999013,0.999938,0.175342,0.999753,low,cad
3,0.998853,0.999881,0.347945,0.999630,low,cad
4,0.998685,0.999815,0.536986,0.999506,low,cad
...,...,...,...,...,...,...
44255,0.934004,0.945113,16.610959,0.939806,high,cvd_death
44256,0.934004,0.945113,16.613699,0.939806,high,cvd_death
44257,0.934004,0.945113,16.616438,0.939806,high,cvd_death
44258,0.934004,0.945113,16.627397,0.939806,high,cvd_death


### Log-rank test

In [12]:
outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    mask = (survival_prob['outcome'] == outcome)
    results = multivariate_logrank_test(survival_prob[mask]['time'], 
                                        survival_prob[mask]['proscore_group'], 
                                        survival_prob[mask]['event'], weightings='wilcoxon')
    data = {
        'outcome': outcome,
        'score': 'proscore',
        'p': results.p_value,
        'test_statistic': results.test_statistic
    }
    data = pd.DataFrame([data])
    
    logrank_df.append(data)
    
logrank_df = pd.concat(logrank_df).reset_index(drop=True)
logrank_df.to_csv('logrank_test.csv', index=False)

## Hazard ratio

In [13]:
hr_df = []
outcomes_list = ["cad", "stroke", "hf", "af", "va", "aaa", "pad", "vt", "cvd_death"]

for outcome in outcomes_list:
    subset = survival_prob[survival_prob['outcome'] == outcome][['proscore_group', 'time', 'event']]
    subset = pd.get_dummies(subset, columns=['proscore_group'], prefix='proscore', drop_first=False)
    subset.drop(columns=['proscore_low'], inplace=True)
    
    cox = CoxPHFitter(penalizer=0.03)
    cox.fit(subset, duration_col='time', event_col='event')
    data = cox.summary.reset_index().rename(columns={
        'index': 'variable',
        'coef': 'coef',
        'exp(coef)': 'exp(coef)',
        'se(coef)': 'se(coef)',
        'coef lower 95%': 'coef lower 95%',
        'coef upper 95%': 'coef upper 95%',
        'exp(coef) lower 95%': 'exp(coef) lower 95%',
        'exp(coef) upper 95%': 'exp(coef) upper 95%',
        'z': 'z',
        'p': 'p',
        '-log2(p)': '-log2(p)'
    })
    data['outcome'] = outcome

    hr_df.append(data)

hr_df = pd.concat(hr_df).reset_index(drop=True)
hr_df.to_csv('survival_hazard_ratio_pro.csv', index=False)
hr_df

,covariate,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p),outcome
0,proscore_high,0.991942,2.696466,0.044840,0.904058,1.079826,2.469603,2.944169,0.0,22.121896,1.945748e-108,357.807909,cad
1,proscore_medium,0.129635,1.138413,0.049706,0.032214,0.227056,1.032739,1.254901,0.0,2.608061,9.105677e-03,6.779018,cad
2,proscore_high,0.511105,1.667132,0.059036,0.395396,0.626814,1.484972,1.871637,0.0,8.657487,4.822794e-18,57.524837,stroke
3,proscore_medium,0.018442,1.018613,0.061585,-0.102262,0.139146,0.902793,1.149292,0.0,0.299457,7.645912e-01,0.387239,stroke
4,proscore_high,0.956823,2.603413,0.054996,0.849032,1.064615,2.337384,2.899721,0.0,17.397910,8.556910e-68,222.794021,hf
5,proscore_medium,-0.121510,0.885582,0.060915,-0.240901,-0.002119,0.785919,0.997883,0.0,-1.994748,4.607038e-02,4.440017,hf
6,proscore_high,1.047835,2.851471,0.047096,0.955528,1.140142,2.600042,3.127213,0.0,22.248710,1.160832e-109,361.875003,af
7,proscore_medium,0.130374,1.139254,0.052214,0.028037,0.232711,1.028434,1.262016,0.0,2.496935,1.252720e-02,6.318792,af
8,proscore_high,0.328404,1.388750,0.068312,0.194515,0.462292,1.214722,1.587710,0.0,4.807420,1.528903e-06,19.319072,va
9,proscore_medium,-0.055435,0.946073,0.069606,-0.191860,0.080989,0.825423,1.084359,0.0,-0.796417,4.257897e-01,1.231787,va
